<h1 style="text-align:center"><strong>Price Elasticity of Demand</strong></h1>

<h2><strong>Problem Statement</strong></h2>


<pre><code>ABC Electronics is a company that produces and sells a variety of electronics goods, including speakers, laptops, and televisions. The company is facing a major challenge in managing their inventory. Despite their efforts to reduce costs and increase sales, they have noticed a decline in sales for a particular category of products. They believe that the decline in sales is due to a change in consumer behaviour, specifically in the way that consumers are responding to changes in price. In order to address this issue, ABC Electronics has asked us to perform a data science project on the Price Elasticity of Demand for their products. This project will be developed with stakeholder"s needs in mind and potentially deliver significant value to sales, marketing, and regulatory managers, by providing data-driven insights into consumer behaviour and product demand.</code></pre>


In [1]:
# IMPORTING PACKAGES
# ------------------------------------------------------
import pandas as pd
import numpy as np
import os
import re
import warnings

# VISUALIZATION
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = 0
pd.options.display.max_colwidth =10

# CONFIGURATIONS
# ------------------------------------------------------
warnings.filterwarnings("ignore")

<h2><strong>Data Gathering</strong></h2>


<pre><code>ABC Electronics has provided us with sales data for the past three years, including information on product categories, prices, and quantities sold.</code></pre>

In [2]:
#os.chdir("C:\\Users\\SkyLake\\Documents\\Python2.0\\01-ML-PEoD\\data")
os.chdir("data/")
raw_data = "G1-PriceElasticityOfDemand-FinalDataSetCombined.csv"

In [4]:
df = pd.read_csv(raw_data)

In [5]:
df.head(1)

,id,prices.amountMax,prices.amountMin,prices.availability,prices.condition,prices.currency,prices.dateSeen,prices.isSale,prices.merchant,prices.shipping,prices.sourceURLs,Cluster,Category_name,asins,brand,categories,dateAdded,dateUpdated,ean,imageURLs,keys,manufacturer,manufacturerNumber,name,primaryCategories,sourceURLs,upc,weight,shipping,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32
0,AVphzg...,104.99,104.99,Yes,New,USD,2017-0...,False,Bestbu...,NaN,http:/...,11,"mount,...",B00C78...,Sanus,Audio ...,2015-0...,2018-0...,NaN,https:...,sanusv...,NaN,VLF410B1,Sanus ...,Electr...,https:...,7.94E+11,32.8 p...,NaN,NaN,NaN,NaN,NaN


In [6]:
# some general information 
print(f"Total number of rows : = {df.shape[0]}")
print(f"Total number of columns : = {df.shape[1]}")

Total number of rows : = 14592
Total number of columns : = 33


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14592 entries, 0 to 14591
Data columns (total 33 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   14592 non-null  object 
 1   prices.amountMax     14592 non-null  float64
 2   prices.amountMin     14592 non-null  float64
 3   prices.availability  12667 non-null  object 
 4   prices.condition     13189 non-null  object 
 5   prices.currency      14592 non-null  object 
 6   prices.dateSeen      14592 non-null  object 
 7   prices.isSale        14592 non-null  bool   
 8   prices.merchant      13820 non-null  object 
 9   prices.shipping      8793 non-null   object 
 10  prices.sourceURLs    14592 non-null  object 
 11  Cluster              14592 non-null  int64  
 12  Category_name        14592 non-null  object 
 13  asins                14592 non-null  object 
 14  brand                14592 non-null  object 
 15  categories           14592 non-null 

<h2><strong>Data Cleaning/Preprocessing</strong></h2>

In [8]:
#Selecting only the columns we require 
df = df.drop(df.columns[-4:], axis=1)
#Removing unnecessary columns
df = df.drop(["primaryCategories","imageURLs","sourceURLs","ean","asins","upc","keys"],axis=1)

In [9]:
#Counting the total imp of product
def imp_date(x1):
    x1_list =x1.strip("'").split(",")
    return x1_list

def imp_counter(x1):
    x1_list =x1.strip("'").split(",")
    x1_count = len(x1_list)
    return x1_count


df["Date"]= df["prices.dateSeen"].apply(imp_date)
df["Imp_count"]= df["prices.dateSeen"].apply(imp_counter)

df =df.explode("Date")

In [10]:
df["prices.availability"].unique()

array(['Yes', 'In Stock', 'TRUE', 'undefined', 'yes', 'Out Of Stock',
       'Special Order', 'No', 'More on the Way', 'sold', 'FALSE',
       'Retired', '32 available', '7 available', nan, 'Limited Stock',
       'Discontinued', 'More Coming Soon'], dtype=object)

In [11]:
# Price availability is categorized
df["prices.availability"] = np.where(df["prices.availability"].str.contains("Yes|TRUE|In Stock|yes|available",case=False),"Yes",
                            np.where(df["prices.availability"].str.contains("Discontinued|Retired|FALSE|No|sold",case=False),"No",
                            np.where(df["prices.availability"].str.contains("Yes|In Stock",case=False),"Special",np.NaN)))

In [12]:
#Product condition categorization
df["prices.condition"] = np.where(df["prices.condition"].str.contains("new",case=False),"New",
                         np.where(df["prices.condition"].str.contains("refurbished",case=False),"Refurbished",
                         np.where(df["prices.condition"].str.contains("pre-owned|used",case=False),"Used",np.NaN)))

In [13]:
#Seller Names
SURL = df["prices.sourceURLs"]
df['Merchant'] = pd.np.where(SURL.str.contains('bhphotovideo', case=False), "bhphotovideo.com",
                 pd.np.where(SURL.str.contains('eBay|e bay', case=False), "ebay.com",
                 pd.np.where(SURL.str.contains('Amazon',case=False), "Amazon.com", 
                 pd.np.where(SURL.str.contains('Bestbuy',case=False), "Bestbuy.com",
                 pd.np.where(SURL.str.contains('Homedepot',case=False), "homedepot.com",
                 pd.np.where(SURL.str.contains('newegg',case=False), "newegg.com",
                 pd.np.where(SURL.str.contains('kmart',case=False), "kmart.com",
                 pd.np.where(SURL.str.contains('frys',case=False), "frys.com",
                 pd.np.where(SURL.str.contains('cdw',case=False), "cdw.com",
                 pd.np.where(SURL.str.contains('target',case=False), "target.com",
                 pd.np.where(SURL.str.contains('overstock',case=False), "overstock.com",
                 pd.np.where(SURL.str.contains('barcodable',case=False), "barcodable.com",
                 pd.np.where(SURL.str.contains('kohls',case=False), "kohls.com",
                 pd.np.where(SURL.str.contains('sears',case=False), "sears.com",
                 pd.np.where(SURL.str.contains('Wal-mart|Walmart',case=False), "Walmart.com","Other")))))))))))))))

In [14]:
df["Merchant"].value_counts().head(15)

Bestbuy.com         16709
bhphotovideo.com    10260
Walmart.com          6179
ebay.com             4406
Amazon.com           2381
target.com           2220
Other                 285
kohls.com             248
barcodable.com        138
frys.com              126
kmart.com             115
cdw.com                79
newegg.com             46
homedepot.com          27
sears.com              10
Name: Merchant, dtype: int64

In [15]:
#Renaming Columns for ease of understanding
df = df.rename(columns={"prices.amountMax":"Actual_price",
                        "prices.amountMin":"Discount_price",
                        "prices.availability":"Product_availability",
                        "prices.condition":"Condition",
                        "prices.isSale":"isSale",
                        "categories":"Description",
                        "brand":"Brand",
                        "name":"Name"})

In [16]:
#Product is on sale or not checker
df['isSale'] = np.where(df['Actual_price']> df['Discount_price'],"Yes", "No")

In [17]:
###################### DATA FILTERING #########################
#setting some filters for major values
dfx = df[ (df["prices.currency"])=="USD"]
dfx = dfx[ (dfx["Condition"])=="New"]

In [18]:
dfx["Condition"].unique()

array(['New'], dtype=object)

In [19]:
#Percentage Discount
dfx["Discount_per"] = round((((dfx["Actual_price"]-dfx["Discount_price"])/dfx["Actual_price"])*100))


In [20]:
#Selecting only the columns we require 
dfx = dfx[["id","Name","Brand","Description","Category_name","Merchant","Actual_price","Discount_price","isSale","Discount_per","Imp_count","Date"]]

In [21]:
#Spliting date to > day, month, week, month name etc.
dfx['Date']= pd.to_datetime(dfx['Date'])
dfx['Day'] = dfx['Date'].dt.day
dfx['Month'] = dfx['Date'].dt.month
dfx['Year'] = dfx['Date'].dt.year
dfx['Week'] = dfx['Date'].dt.week
dfx['Month_Name'] = dfx['Date'].dt.strftime('%B')
dfx['Day_Name'] = dfx['Date'].dt.strftime('%A')
dfx['Year_Month'] = dfx['Date'].dt.strftime('%Y-%m')
dfx['Year_Week'] = dfx['Date'].dt.strftime('%Y,%W')
dfx['date'] = pd.to_datetime(dfx['Date'].dt.date)
dfx = dfx.drop(columns=["Date"])
dfx = dfx.rename(columns={"date":"Date"})

In [22]:
dfx = dfx.reset_index(drop=True)

In [23]:
dfx.sort_values("Date")

,id,Name,Brand,Description,Category_name,Merchant,Actual_price,Discount_price,isSale,Discount_per,Imp_count,Day,Month,Year,Week,Month_Name,Day_Name,Year_Month,Year_Week,Date
31798,AVpfnL...,Sony -...,Sony,Used:A...,camera...,Amazon...,59.99,59.99,No,0.0,1,30,5,2014,22,May,Friday,2014-05,"2014,21",2014-05-30
31797,AVpfnL...,Sony -...,Sony,Used:A...,camera...,Amazon...,29.39,29.39,No,0.0,1,30,5,2014,22,May,Friday,2014-05,"2014,21",2014-05-30
35163,AVpfcu...,Canon ...,Canon,Electr...,camera...,Amazon...,1048.00,1048.00,No,0.0,1,23,8,2014,34,August,Saturday,2014-08,"2014,33",2014-08-23
17638,AVpgzp...,AudioQ...,AudioQ...,Speake...,"cable,...",Bestbu...,179.99,179.99,No,0.0,1,30,8,2014,35,August,Saturday,2014-08,"2014,34",2014-08-30
27838,AVpf6Q...,Insign...,Insignia™,Outdoo...,speake...,Bestbu...,29.99,29.99,No,0.0,4,31,8,2014,35,August,Sunday,2014-08,"2014,34",2014-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30987,AV1T0K...,Magic ...,Apple,Comput...,keyboa...,bhphot...,99.99,99.99,No,0.0,4,25,7,2018,30,July,Wednesday,2018-07,"2018,30",2018-07-25
35360,AV1YHu...,Alpha ...,Sony,Digita...,camera...,bhphot...,996.00,996.00,No,0.0,7,25,7,2018,30,July,Wednesday,2018-07,"2018,30",2018-07-25
23188,AVpf64...,1506TS...,Pelican,Comput...,tablet...,bhphot...,12.95,12.95,No,0.0,6,25,7,2018,30,July,Wednesday,2018-07,"2018,30",2018-07-25
5594,AVpfB5...,Audio ...,SKB,Comput...,microp...,bhphot...,64.99,64.99,No,0.0,3,25,7,2018,30,July,Wednesday,2018-07,"2018,30",2018-07-25


In [24]:
dfx

,id,Name,Brand,Description,Category_name,Merchant,Actual_price,Discount_price,isSale,Discount_per,Imp_count,Day,Month,Year,Week,Month_Name,Day_Name,Year_Month,Year_Week,Date
0,AVphzg...,Sanus ...,Sanus,Audio ...,"mount,...",Bestbu...,104.99,104.99,No,0.0,4,30,3,2017,13,March,Thursday,2017-03,"2017,13",2017-03-30
1,AVphzg...,Sanus ...,Sanus,Audio ...,"mount,...",Bestbu...,104.99,104.99,No,0.0,4,10,3,2017,10,March,Friday,2017-03,"2017,10",2017-03-10
2,AVphzg...,Sanus ...,Sanus,Audio ...,"mount,...",Bestbu...,104.99,104.99,No,0.0,4,4,3,2017,9,March,Saturday,2017-03,"2017,09",2017-03-04
3,AVphzg...,Sanus ...,Sanus,Audio ...,"mount,...",Bestbu...,104.99,104.99,No,0.0,4,3,3,2017,9,March,Friday,2017-03,"2017,09",2017-03-03
4,AVpgMu...,Boyton...,Boytone,Stereo...,speake...,Walmar...,69.00,64.99,Yes,6.0,1,14,12,2017,50,December,Thursday,2017-12,"2017,50",2017-12-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41864,AVpgib...,naxa -...,naxa,TVs & ...,"sound,...",Bestbu...,58.99,58.99,No,0.0,6,1,6,2017,22,June,Thursday,2017-06,"2017,22",2017-06-01
41865,AVpgib...,naxa -...,naxa,TVs & ...,"sound,...",Bestbu...,58.99,58.99,No,0.0,6,1,6,2017,22,June,Thursday,2017-06,"2017,22",2017-06-01
41866,AVpgib...,naxa -...,naxa,TVs & ...,"sound,...",Walmar...,58.49,58.49,No,0.0,1,5,3,2018,10,March,Monday,2018-03,"2018,10",2018-03-05
41867,AVpgib...,naxa -...,naxa,TVs & ...,"sound,...",Walmar...,77.98,77.98,No,0.0,1,22,3,2016,12,March,Tuesday,2016-03,"2016,12",2016-03-22


In [25]:
#Exporing the Cleaning Data as CSV
dfx.to_csv("CleanedData.csv",index=False)